# Importing the Data

In [130]:
from xml.etree import ElementTree
from xml.dom import minidom
import os
import cv2
import numpy as np
from pandas import *

In [131]:
xmldoc = minidom.parse('MiracleDataFile.xml')
itemlist = xmldoc.getElementsByTagName('word')

    # In[67]:


    #images_labels = []
    #images_labels.append([]) #making a 2D array in python
    #images_labels.append([])
    
tags = []
words = []
for z in itemlist:
    tags.append(z.attributes['id'].value)
    words.append(z.attributes['text'].value)

    #print(store)
df = DataFrame({'Word Id': tags, 'Text': words})
df = DataFrame.drop_duplicates(df)
df = df.sort_values(by = ['Word Id'], ascending=[True])

In [132]:
labels = [] 
imgs = []

image_id_real = []

for file in os.listdir('C:/Users/Adrian/Desktop/SparkDevAI2018/TestSetWords/TestSetWords'):
        for file2 in os.listdir('C:/Users/Adrian/Desktop/SparkDevAI2018/TestSetWords/TestSetWords/' + file):
            for image in os.listdir('C:/Users/Adrian/Desktop/SparkDevAI2018/TestSetWords/TestSetWords/' + file + '/' + file2):
                imgs.append(cv2.imread('C:/Users/Adrian/Desktop/SparkDevAI2018/TestSetWords/TestSetWords/' + file + '/' + file2+ '/'+image))
                image_id = image.split('.')
                image_id_real.append(image_id[0])

In [133]:
for index in image_id_real:
    row = df[df['Word Id'].str.match(index)]
    word = row.iloc[0]['Text']
    labels.append(word)

In [172]:
letters = ''

for x in labels:
    letters += x

letters = "".join(set(letters))
letters = "".join(sorted(letters))

print(len(letters))


70


In [135]:
def labels_to_text(labels):
    return ''.join(list(map(lambda x: letters[int(x)], labels)))

def text_to_labels(text):
    return list(map(lambda x: letters.index(x), text))


In [212]:
img_data = []

for img in imgs:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (128, 64))
    data = np.asarray(img, dtype = "int32")
    #data /= 255
    img_data.append(data)

img_data = np.array(img_data)
print(img_data.shape)

(4824, 64, 128)


In [213]:
labels_encoded = []
for word in labels:
    labels_encoded.append(text_to_labels(word))
labels_encoded = np.array(labels_encoded) #object)

In [214]:
print(len(max(labels_encoded, key = tuple)))

9


# Splitting into X and Y Data

In [215]:
from sklearn.model_selection import train_test_split

In [236]:
X, y = img_data, labels_encoded
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .15)

# Making The Model

In [217]:
from keras.models import Sequential
from keras.layers import Conv2D #convolution 2d for images
from keras.layers import MaxPooling2D #for pooling
from keras.layers import Flatten #for flattening the pooling into the inputs of full layer
from keras.layers import Dense #to create hidden layers, into ann
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Dropout
from keras.layers import Reshape
from keras.layers import Lambda
from keras.layers import Bidirectional
from keras.layers import Activation
from keras.layers import Input
from keras.layers.merge import add, concatenate
from keras import backend as K
from keras.optimizers import SGD
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.models import Model, load_model

In [218]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [219]:
def functional():
    img_h = 64
    img_w = 128
    # Network parameters
    conv_filters = 16
    kernel_size = (3, 3)
    pool_size = 2
    time_dense_size = 32
    rnn_size = 512
        
    batch_size = 32
    downsample_factor = pool_size ** 2

    act = 'relu'
    input_data = Input(name='the_input', shape=(img_w, img_h, 1), dtype='float32')
    inner = Conv2D(conv_filters, kernel_size, padding='same',
                   activation=act, kernel_initializer='he_normal',
                   name='conv1')(input_data)
    inner = MaxPooling2D(pool_size=(pool_size, pool_size), name='max1')(inner)
    inner = Conv2D(conv_filters, kernel_size, padding='same',
                   activation=act, kernel_initializer='he_normal',
                   name='conv2')(inner)
    inner = MaxPooling2D(pool_size=(pool_size, pool_size), name='max2')(inner)

    conv_to_rnn_dims = (img_w // (pool_size ** 2), (img_h // (pool_size ** 2)) * conv_filters)
    inner = Reshape(target_shape=conv_to_rnn_dims, name='reshape')(inner)

    # cuts down input size going into RNN:
    inner = Dense(time_dense_size, activation=act, name='dense1')(inner)

    # Two layers of bidirecitonal GRUs
    # GRU seems to work as well, if not better than LSTM:
    gru_1 = GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', name='gru1')(inner)
    gru_1b = GRU(rnn_size, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru1_b')(inner)
    gru1_merged = add([gru_1, gru_1b])
    gru_2 = GRU(rnn_size, return_sequences=True, kernel_initializer='he_normal', name='gru2')(gru1_merged)
    gru_2b = GRU(rnn_size, return_sequences=True, go_backwards=True, kernel_initializer='he_normal', name='gru2_b')(gru1_merged)

    # transforms RNN output to character activations:
    inner = Dense(71, kernel_initializer='he_normal',
                  name='dense2')(concatenate([gru_2, gru_2b]))
    y_pred = Activation('softmax', name='softmax')(inner)
    Model(inputs=input_data, outputs=y_pred).summary()

    labels = Input(name='the_labels', shape=[9], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64')
    # Keras doesn't currently support loss funcs with extra parameters
    # so CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

    # clipnorm seems to speeds up convergence
    sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)

    model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)

    # the loss calc occurs elsewhere, so use a dummy lambda func for the loss
    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=sgd)
    
    
    # captures output of softmax so we can decode the output during visualization

    return model

In [241]:
def model(img_w, img_h, max_text_len):
    #Parameters
    conv_filters = 16
    kernel_size = (3, 3)
    pool_size = 2
    time_dense_size = 32
    rnn_size = 512
    #Initializing CNN
    classifier = Sequential()
    
    #Step 1
    classifier.add(Convolution2D(conv_filters, kernel_size, input_shape=(img_w, img_h, 1), padding = 'same', activation="relu",  kernel_initializer='he_normal'))
    
    print(classifier.layers[0].shape)
    #Step2 MAXadd(Convolution2D(32, (3, 3), activation="relu"))
    #POOLING
    classifier.add(MaxPooling2D(pool_size=(2, 2)))

    #to make more accurate results, another convolutional layer.
    classifier.add(Convolution2D(conv_filters, kernel_size, padding = 'same', activation="relu", kernel_initializer='he_normal'))
    classifier.add(MaxPooling2D(pool_size=(2, 2)))
    
    conv_to_rnn_dims = (img_w // (pool_size ** 2), (img_h // (pool_size ** 2)) * conv_filters)
    
    #print(conv_to_rnn_dims)
    
    classifier.add(Reshape(target_shape=conv_to_rnn_dims))
    
    #1 par how many neurons in hidden layer 128 gotten from experimentation more than 100
    classifier.add(Dense(units = 32, activation = 'relu'))
        
    # Adding the first LSTM layer and some Dropout regularisation
    classifier.add(Bidirectional(LSTM(units = 512, return_sequences = True, kernel_initializer='he_normal')))
    classifier.add(Dropout(0.2))
    
    # Adding a second LSTM layer and some Dropout regularisation
    classifier.add(Bidirectional(LSTM(units = 512, return_sequences = True, kernel_initializer='he_normal')))
    classifier.add(Dropout(0.2))
    
    classifier.add(Flatten())
    # Adding the output layer
    classifier.add(Dense(units = len(letters)+1, activation = 'softmax', kernel_initializer='he_normal'))
    #classifier.add(Activation('softmax'))
    
    #labels = Input(name='the_labels', shape=[max_text_len], dtype='float32')
    #print(labels)
    #input_length = Input(name='input_length', shape=[1], dtype='int64')
    #print(input_length)
    #label_length = Input(name='label_length', shape=[1], dtype='int64')
    #print(label_length)
    
    
    print(classifier.summary())
    
    #intermediate_layer_model = Model(inputs=classifier.input, outputs=classifier.layers[11].output)
    #y_pred = classifier.layers[11].get_output_at(0)

    #y_pred = classifier.layers[11].get_output_at(0)

    #print(y_pred)
    
    #classifier.add(Lambda(ctc_lambda_func(y_pred, labels, input_length, label_length), output_shape=(1,)))
    
    sgd = SGD(lr=0.02, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5)
    
    #model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
    # Compiling
    #classifier.compile(loss= {'ctc': lambda y_true, y_pred: y_pred}, optimizer= sgd)
    classifier.compile(loss = 'categorical_crossentropy', optimizer = sgd)
    return classifier

# Let's Test

In [242]:
model = model(128, 64, 9)

AttributeError: 'Conv2D' object has no attribute 'shape'

In [243]:
print (X_train.shape)
X_train = np.reshape(X_train, (4100, 128, 64, 1))

#y_train = np.array(y_train)
#print(X_train)

model.fit(X_train, y_train, batch_size=1, epochs=1)

(4100, 128, 64, 1)


AttributeError: 'function' object has no attribute 'fit'